In [129]:
# d for coin futures, f for usd s futures
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import requests as re
import pandas as pd
from functools import reduce

api = 'https://api.binance.com/api/v3/klines'
g_api_key='eJBYUpKfnUCA5wXDfJTKGPNaOoyLtnXIb9D6GKFbnABLCndVRRYJrdyI9rMi6D54'
g_secret_key='970sbCiGtH1W4rPW56I4MBbD9brKZegyveedljt3U3D5hry7zMbxa4sxacA80kUv'
headers = {"Accept": "application/json"}

epoch = datetime.utcfromtimestamp(0)

def dt_to_ms(dt):
    delta = dt - epoch
    return int(delta.total_seconds() * 1000)

def dt_from_ms(ms):
    return datetime.utcfromtimestamp(ms / 1000.0)

def json_to_df(json):
    df = {'time':[],'open':[],'high':[],'low':[],'close':[],'volume':[]}
    for item in json:
        df['time'].append(dt_from_ms(item[0]))
        df['open'].append(float(item[1]))
        df['high'].append(float(item[2]))
        df['low'].append(float(item[3]))
        df['close'].append(float(item[4]))
        df['volume'].append(float(item[5]))
    return pd.DataFrame(df)

In [130]:
# save down data
currTime = dt_to_ms(datetime.now())
fpath = './data/'
pair = 'LTCUSDT'
interval = '1h'
start_date = '20210101'
start_year, start_month, start_day = int(start_date[0:4]), int(start_date[4:6]), int(start_date[6:8])
start_date = dt_to_ms(datetime(start_year, start_month, start_day))
limit = 1000
dfs = []
while start_date < currTime:
    url = api + '?symbol='+pair+'&interval='+interval+'&startTime='+str(start_date)+'&limit='+str(limit)
    r = re.get(url, headers=headers)
    result = r.json()
    df = json_to_df(result)
    dfs.append(df)
    start_date = dt_to_ms(pd.to_datetime(df.tail(1)['time'].values[0])) + 1*60*60*1000
df_tot = reduce(lambda x,y: x.append(y), dfs)
df_tot = df_tot.sort_values('time')
df_tot.set_index('time',inplace=True)
df_tot.to_csv(fpath+pair+'.csv')

In [131]:
import sys
import logging

class Algo:

    def init(self):
        self.logger = logging.getLogger('Algo')
        self.keys = []
        self.data = []
        return

    def ParseConfig(self, config):
        """ Parse Input Config """

        ##symbols are formated like EXCHANGE-SYMBOL (exchange specific grammar)
        self.symbol = config["symbol"]
        #self.fields = config["fields"]
        self.sdt = config["sdt"]
        self.edt = config["edt"]
        self.data_dir = config["data_dir"]
        #self.file_prefix = config['file_prefix']
        #self.data_index = config['data_index']
        #self.data_loader = DataLoader(self.symbols, self.fields)
        self.logger.info('Config Parsed')

        return

    def GetData(self):
        """ Collect Data -- According to specified symbols and fields """
        df = pd.read_csv(self.data_dir+'/'+self.symbol+'.csv')
        df['time'] = pd.to_datetime(df['time'])
        df.set_index('time', inplace=True)
        self.data = df
        #self.data = self.data_loader.GetData(self.data_dir, self.file_prefix, self.data_index, self.sdt, self.edt)
        #TODO -- Implement date filter later
        # self.data = self.data[self.data['dt'] >= self.sdt]
        # self.data = self.data[self.data['dt'] <= self.edt]

        #self.keys = sorted(list(self.data.keys())[1:])
        #self.labels = []
        #for key in self.keys:
        #    self.labels.append(key.split(":")[1].split("@")[0])

        self.logger.info('Data Grab Successful')

        return

    def GenerateSignal(self):
        """ Signal Generation is Strategy Specific """

        pass

    def ConstructPortfolio(self):
        """ Position Construction is Strategy Specific """

In [132]:
from numpy.random import normal
import numpy as np
import pandas as pd
import math

def price_simulation(start_value, drift, diffusion, n_periods):
    returns = [1 + (drift + diffusion * normal()) for _ in range(n_periods)]
    prices = [start_value * np.prod(returns[:i]) for i in range(len(returns))]
    return (returns, prices)

def price_simulation_gbm(start_value, drift, diffusion, n_periods):
    prices = []
    for _ in range(n_periods):
        start_value *= math.exp((drift - (diffusion**2) / 2) + diffusion * normal())
        prices.append(start_value)
    return prices

def monte_carlo_simulation(start_value, drift, diffusion, n_periods, n_simulations):
    results = []
    for _ in range(n_simulations):
        prices = price_simulation_gbm(start_value, drift, diffusion, n_periods)
        results.append(prices)
    terminal_values = [x[-1] for x in results]
    return terminal_values

def get_bounds(values, pct):
    diff = 1. - pct
    deciles = pd.qcut(values, [diff / 2, 1-(diff / 2)])[-1]
    return (deciles.left, deciles.right)

def save_positions(df, columns, data):
    for i in range(len(columns)):
        df[columns[i]].append(data[i])
    return df

In [148]:
from datetime import datetime
import logging, sys, numpy as np, pandas as pd
import scipy.stats as st

class UniswapV3(Algo):

    def __init__(self):

        super().__init__()
        self.logger = logging.getLogger("UniswapV3")

        return

    def ParseConfig(self, config):
        """ Parse Input Config """

        super().ParseConfig(config)
        self.rolling_window = config["rolling_window"]
        self.does_stop = config["does_stop"]
        self.stop_pct = config["stop_pct"]
        self.size = config["size"]
        self.n_simulations = config["n_simulations"]
        self.n_periods = config["n_periods"]
        self.pct = config["pct"]
        return

    def GenerateSignal(self):
        """ Generate Signal for input to Portfolio Construction """
        
        self.logger.info("Generating signal")
        data = self.data
        data['returns'] = data['close'].pct_change(1)
        data['rolling_mean'] = data['returns'].rolling(self.rolling_window).mean()
        data['rolling_std'] = data['returns'].rolling(self.rolling_window).std()
        self.data = data
        pass

        return

    def GeneratePositions(self):
        """ Generate Signal for input to Portfolio Construction """

        self.logger.info("Generating positions")
        # Downsample data according to config param
        
        positions = {'time':[],'type':[],'low':[],'high':[], 'amount':[], 'note':[]}
        start_date = self.sdt
        end_date = dt_from_ms(dt_to_ms(self.sdt) + 24*60*60*1000)
        stop_left, stop_right = np.nan, np.nan
        for i in range(self.data.shape[0]):
            date = self.data.index[i]
            close = self.data.iloc[i]['close']
            if date < end_date:
                # check if stopping out
                if ((close > stop_right) or (close < stop_left)):
                    px = self.data.iloc[i]['close']
                    drift = self.data.iloc[i]['rolling_mean']
                    diffusion = self.data.iloc[i]['rolling_std']
                    terminal_values = monte_carlo_simulation(px, drift, diffusion, self.n_periods, self.n_simulations)
                    left, right = get_bounds(terminal_values, self.pct)
                    positions = save_positions(positions, ['time','type','low','high','amount','note'], [date, 'base', left, right, self.size, 'stop'])
                    end_date = dt_from_ms(dt_to_ms(date) + 24*60*60*1000)
                    if self.does_stop:
                        stop = (1-self.stop_pct) / 2
                        stop_left, stop_right = left + stop * (right-left), right - stop * (right-left)
            elif date == end_date:
                # set new position
                px = self.data.iloc[i]['close']
                drift = self.data.iloc[i]['rolling_mean']
                diffusion = self.data.iloc[i]['rolling_std']
                terminal_values = monte_carlo_simulation(px, drift, diffusion, self.n_periods, self.n_simulations)
                left, right = get_bounds(terminal_values, self.pct)
                positions = save_positions(positions, ['time','type','low','high','amount','note'], [date, 'base', left, right, self.size, 'new'])
                end_date = dt_from_ms(dt_to_ms(date) + 24*60*60*1000)
                if self.does_stop:
                    stop = (1-self.stop_pct) / 2
                    stop_left, stop_right = left + stop * (right-left), right - stop * (right-left)
            else:
                print("data issue")

        positions = pd.DataFrame(positions)
        self.positions = positions
        #results = pd.DataFrame(data=results, columns=[self.data_index])
        #self.data = self.data.set_index([self.data_index]).join(results.set_index([self.data_index])).reset_index()
        #positions = np.array(positions)
        #self.data[[col +"@position" for col in self.symbols]] = positions

        return


if __name__ == "__main__":

    logging.basicConfig()
    logging.getLogger().setLevel(logging.INFO)

    algo = UniswapV3()
    data_dir = "./data"
    positions_dir = "./data/position"
    #exchange = "binance-futures"
    #file_prefix = "derivative_ticker"

    #coins = market_coins[key]
    algo.ParseConfig({"symbol": 'ETHUSDT',
                      "sdt": datetime(2021, 4, 1),  # start date
                      "edt": datetime(2021, 5, 1),  # end date
                      "size": 1,
                      "data_dir": data_dir,
                      "rolling_window": 24*30,
                      "does_stop": True,
                      "stop_pct": 0.95,
                      "n_simulations": 10000,
                      "n_periods": 24,
                      "pct": 0.95
                      })

    algo.GetData()
    #algo.ConstructPortfolio()

    #algo.data = PnlCalculator().CalculatePnL(algo.data, algo.symbols)
    #algo.data.to_csv(positions_dir + "/MeanReversionv1_test_" + key + ".csv")


INFO:UniswapV3:Config Parsed
INFO:UniswapV3:Data Grab Successful


In [149]:
algo.GenerateSignal()
algo.GeneratePositions()

INFO:UniswapV3:Generating signal
INFO:UniswapV3:Generating positions


In [150]:
algo.positions

,time,type,low,high,amount,note
0,2021-04-02 00:00:00,base,1824.423,2141.635,1,new
1,2021-04-03 00:00:00,base,1980.237,2325.723,1,new
2,2021-04-04 00:00:00,base,1870.488,2178.257,1,new
3,2021-04-05 00:00:00,base,1923.986,2242.371,1,new
4,2021-04-06 00:00:00,base,2007.059,2326.603,1,new
5,2021-04-07 00:00:00,base,1953.899,2270.412,1,new
6,2021-04-08 00:00:00,base,1851.755,2144.072,1,new
7,2021-04-09 00:00:00,base,1954.136,2261.893,1,new
8,2021-04-10 00:00:00,base,1936.241,2233.985,1,new
9,2021-04-11 00:00:00,base,2001.682,2302.388,1,new
